# Exploration of Aerosol Optical Depth as a VCI Metric

This notebook looks at the possibility of using aerosol optical depth as a metric (or component of a metric) to estimate "Volcanic Climate Index" (VCI). The goal is to answer questions such as:

1. How senstive is the metric to choices of parameters (time, altitude integration, etc...)
2. Can the background aerosol be removed?
3. Is clustering of volcanoes a problem?
4. How would a metric such as this differ from a temperature based one?

In [176]:
from bokeh.io import output_notebook
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, LinearColorMapper, ColorBar, Line, Patch, RangeTool, Range1d, Label, Legend, HoverTool
from bokeh.plotting import figure, show
import xarray as xr
import pandas as pd
import numpy as np
output_notebook()

Loading BokehJS ...

## Load in the data

In [204]:
# open the glossac dataset
glossac_file = r'C:\Users\lando\data\netcdf\GloSSAC\GloSSAC-V1.nc'
glossac = xr.open_dataset(glossac_file, decode_times=False)

# read in extinction at 1020nm
extinction = glossac.Measurements_extinction
extinction = extinction.rename({'altitude1': 'altitude',
                                'measurement wavelengths': 'ext_wavel', 
                                'years':'time'})\
                       .sel(ext_wavel=5)
time = pd.date_range('1979', '2016-12-31', freq='MS')
extinction.assign_coords(time=time, latitude=glossac.lat.values, altitude=extinction.altitude.values)

# tile the tropropause over the entire dataset
tropopause = xr.DataArray(np.tile(glossac.trop.values, int(len(glossac.years) / 12)).T,
                          dims=['time', 'latitude'],
                          coords=[time, glossac.lat.values])

volcanoes = {'El Chichon': (np.datetime64('1982-04-03'), 17.21, -93.13, 5),
             'Nevado del Ruiz': (np.datetime64('1985-10-11'), 4.53, -79.19, 3),
             'Pinatubo': (np.datetime64('1991-06-15'), 15.08, 120.21, 6),
             'Ruang': (np.datetime64('2002-09-25'), 2.3, 125.37, 4),
             'Reventador': (np.datetime64('2002-11-02'), -0.4, -39.21, 4),
             'Manam': (np.datetime64('2005-01-28'), -4.1, 145, 4),
             'Soufriere Hills': (np.datetime64('2006-05-20'), 16.43, -62.11, 3),
             'Rabaul': (np.datetime64('2006-10-07'), -4.14, 152.12, 4),
             'Okmok': (np.datetime64('2008-07-12'), 53.28, -168, 4),
             'Kasatochi': (np.datetime64('2008-08-07'), 52.1, -175, 4),
             'Redoubt': (np.datetime64('2009-03-15'), 60.29, -152.44, 3),
             'Sarychev': (np.datetime64('2009-06-11'), 48.05, 153, 4),
             'Merapi': (np.datetime64('2010-10-25'), -7.32, 110.26, 4),
             'Nabro': (np.datetime64('2011-06-12'), 13.36, 41.69, 4),
             'Puyehue-Cordon': (np.datetime64('2011-06-04'), -40.35, -72.07, 5),
             'Copahue': (np.datetime64('2012-12-23'), 37.51, -71.1, 2),
             'Kelud': (np.datetime64('2014-02-13'), -7.55, 112, 4),
             'Calbuco': (np.datetime64('2015-04-22'), -41.19, -72.37, 4)}

volcanoes = pd.DataFrame.from_dict(volcanoes, orient='index', columns=['time', 'latitude', 'longitude', 'VEI'])
volcanoes.index.name = 'Volcano'

## Compute Aerosol Optical Depth

In [205]:
# compute aod above the tropopause
aod = extinction.where(extinction.altitude > tropopause).sum(dim='altitude') * 0.5

# integrate globally
global_aod = (np.cos(aod.latitude * np.pi / 180) * aod).sum(dim='latitude') \
             / np.cos(aod.latitude * np.pi / 180).sum(dim='latitude')

# compute the 'volcanically perturbed' aod
background = aod.sel(time=slice('1998', '2002')).groupby('time.month').mean(dim='time')
aod_diff = aod.groupby('time.month') - background
global_aod_diff = (np.cos(aod.latitude * np.pi / 180) * aod_diff).sum(dim='latitude') \
                 / np.cos(aod.latitude * np.pi / 180).sum(dim='latitude')

# make the ColumnDataSources for the bokeh plot
df = pd.DataFrame(aod.to_pandas().stack(), columns=['Measurements_extinction']).reset_index()
df['width'] = pd.Timedelta(1, 'M')
source = ColumnDataSource(df)

aod_df = ColumnDataSource({'time': global_aod.time.values, 'aod': global_aod.values})
aod_diff_df = ColumnDataSource({'time': global_aod_diff.time.values, 'aod_diff': global_aod_diff.values})

volc_df = ColumnDataSource(volcanoes)

## Make the Plots

In [221]:
def modify_doc(doc):

    def calc_vci():
        aod = global_aod_diff.sel(time=slice(date_to_str(range_tool.x_range.start),
                                             date_to_str(range_tool.x_range.end)))
        vci = aod.integrate(dim='time', datetime_unit='D').values * 365
        top = vci - xr.concat([aod.isel(time=0), aod.isel(time=-1)], dim='time')\
                      .integrate(dim='time', datetime_unit='D').values * 365
        return np.log10(top)

    def date_to_str(date):
        try:
            return str(pd.Timedelta(date, 'ms') + pd.Timestamp('1970'))
        except:
            return str(date)

    def vci_callback(attr, old, new):
        label2.text = f'VCI: {calc_vci():.2f}'
        label2.x = np.datetime64(date_to_str(range_tool.x_range.end))
        aod_band_diff2.data = calc_aod_in_period().data

    def calc_aod_in_period():
        aod_sel = global_aod_diff.sel(time=slice(date_to_str(range_tool.x_range.start), 
                                                 date_to_str(range_tool.x_range.end)))
        good = aod_sel > 0
        upper = aod_sel.values[(good,)].flatten()
        aod_time = aod_sel.time.values[(good,)].flatten()
        return ColumnDataSource({'time': aod_time, 'aod_fill': upper})

    # ------------------------------------
    # plot the latitudinally resolved AOD
    # ------------------------------------
    hover = HoverTool(tooltips=[("time", "@time{%F}"), ("latitude", "@latitude"), 
                                ("Volcano", "@Volcano"), ("VEI", "@VEI"), 
                                ("AOD", "@Measurements_extinction"),],
                      formatters={'time': 'datetime'},)
    
    p = figure(plot_height=300, plot_width=900, 
               x_axis_type='datetime', y_range=(-80, 80), 
               x_range=(aod.time.values[0], aod.time.values[-1]), tools=[hover,])
    mapper = LinearColorMapper(palette='Viridis256', low=0, high=0.01)
    rect = p.rect(x="time", y="latitude", width='width', height=np.diff(aod.latitude)[0],
                  source=source,
                  fill_color={'field': 'Measurements_extinction', 'transform': mapper},
                  line_color={'field': 'Measurements_extinction', 'transform': mapper})
    p.yaxis.axis_label = "Latitude"
    color_bar = ColorBar(color_mapper=mapper,
                         label_standoff=12, border_line_color=None, location=(0, 0))
    p.add_layout(color_bar, 'right')
    
    # add the eruption markers
    p.circle('time', 'latitude', size=10, color='gray', source=volc_df, name='volcanoes')

    # ------------------------------------
    # plot the globally averaged AOD
    # ------------------------------------    
    p2 = figure(plot_height=300, plot_width=900, 
                x_axis_type='datetime', y_axis_type='log', 
                y_range=(1e-4, 1e-1), x_range=p.x_range)
    a0 = p2.line(x='time', y='aod', source=aod_df, color='gray')
    a1 = p2.line(x='time', y='aod_diff', source=aod_diff_df, line_width=1)
    p2.yaxis.axis_label = 'Global AOD [1020nm]'

    # ------------------------------------
    # Add the range tool for selection
    # ------------------------------------
    range_tool = RangeTool(x_range=Range1d(np.datetime64('1991-05-01'), np.datetime64('1996-06-01')))
    range_tool.overlay.fill_color = "gray"
    range_tool.overlay.fill_alpha = 0.1

    range_tool.x_range.on_change('start', vci_callback)
    range_tool.x_range.on_change('end', vci_callback)

    p2.ygrid.grid_line_color = None
    p2.add_tools(range_tool)
    p.add_tools(range_tool)
    p2.toolbar.active_multi = range_tool

    aod_band_diff2 = calc_aod_in_period()
    glyph = Patch(x="time", y="aod_fill", fill_color="#f46842", line_width=0, fill_alpha=0.3)
    p2.add_glyph(aod_band_diff2, glyph)

    label2 = Label(y=0.02, x=np.datetime64(date_to_str(range_tool.x_range.end)),
                   text=f'VCI: {calc_vci():.2f}',
                   text_align='center', text_baseline='middle', text_color='#f46842')
    p2.add_layout(label2)

    legend = Legend(items=[("Deseasonalized", [a1]), ("AOD", [a0])], location="center")
    p2.add_layout(legend, 'below')
    p2.legend.orientation = "horizontal"

    doc.add_root(column(p, p2))

## Display the Plot

Move the bounds around to determine the VCI as calculated as the global aerosol optical depth after the background layer has been removed. VCI_1 is the globally integrated sAOD, while VCI_2 subtracts a further baseline off of VCI_1 to try to account for aerosol levels before the eruption.

NOTE: If a different local host is used for the notebook it will be need to be updated in the `show` command

In [222]:
show(modify_doc, notebook_url='localhost:8889') # notebook_url="http://localhost:8889"

C:\Users\lando\Anaconda3\envs\volcanic-climate-impact\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in log10
  if __name__ == '__main__':
C:\Users\lando\Anaconda3\envs\volcanic-climate-impact\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in log10
  if __name__ == '__main__':
C:\Users\lando\Anaconda3\envs\volcanic-climate-impact\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in log10
  if __name__ == '__main__':
C:\Users\lando\Anaconda3\envs\volcanic-climate-impact\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in log10
  if __name__ == '__main__':
C:\Users\lando\Anaconda3\envs\volcanic-climate-impact\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in log10
  if __name__ == '__main__':
C:\Users\lando\Anaconda3\envs\volcanic-climate-impact\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: in